19/06/24 - basics working nicely. Didn't try fit options as yet.

Note CI est. currently seems to be an issue - running for several hours with no output. Maybe better with tighter limits set...?

UPDATE: good parts now distilled to `-redux` version of this notebook.

(page:advanced-fitting)=
# Advanced fitting for hyperfine beat (stage 1 bootstrap)

For advanced fitting, try a stage 1 style bootstrap. In this case, options are:

- "basic" ignore the photoionization dynamics and just try fitting the beat to the $l=4$, ROI=0 case, since it is already pretty close and may be assumed to be directly mapped here. See {ref}`page:advanced-fitting`.
- "advanced" set (arbitrary) parameters per final state for the probe, and fit these plus the hyperfine beat model parameters. This should allow for a match to a single set of hyperfine parameters for all observables, and fulfil the stage 1 bootstrap criteria. (This page.)

---

From prior work and data:

- Forbes, R. et al. (2018) ‘Quantum-beat photoelectron-imaging spectroscopy of Xe in the VUV’, Physical Review A, 97(6), p. 063417. Available at: https://doi.org/10.1103/PhysRevA.97.063417. arXiv: http://arxiv.org/abs/1803.01081, Authorea (original HTML version): https://doi.org/10.22541/au.156045380.07795038
- Data (OSF): https://osf.io/ds8mk/
- [Quantum Metrology with Photoelectrons (Github repo)](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons), particularly the [Alignment 3 notebook](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons/blob/master/Alignment/Alignment-3.ipynb). Functions from this notebook have been incorporated in the current project, under `qbanalysis.hyperfine`.

## Setup fitting model

Follow the modelling notebook ({ref}`page:hyperfine-beat-model`), but wrap functions for fitting.

New functions are in `qbanalysis.adv_fitting.py`.

### Imports

In [1]:
# Load packages
# Main functions used herein from qbanalysis.hyperfine
from qbanalysis.hyperfine import *
import numpy as np
from epsproc.sphCalc import setBLMs

from pathlib import Path

dataPath = Path('/tmp/xe_analysis')
# dataTypes = ['BLMall', 'BLMerr', 'BLMerrCycle']   # Read these types, should just do dir scan here.

# # Read from HDF5/NetCDF files
# # TO FIX: this should be identical to loadFinalDataset(dataPath), but gives slightly different plots - possibly complex/real/abs confusion?
# dataDict = {}
# for item in dataTypes:
#     dataDict[item] = IO.readXarray(fileName=f'Xe_dataset_{item}.nc', filePath=dataPath.as_posix()).real
#     dataDict[item].name = item

# Read from raw data files
from qbanalysis.dataset import loadFinalDataset
dataDict = loadFinalDataset(dataPath)

# Use Pandas and load Xe local data (ODS)
# These values were detemermined from the experimental data as detailed in ref. [4].
from qbanalysis.dataset import loadXeProps
xeProps = loadXeProps()

2024-06-19 13:40:52.240 | INFO     | qbanalysis.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 


* Setting plotter defaults with epsproc.basicPlotters.setPlotters(). Run directly to modify, or change options in local env.


* Set Holoviews with bokeh.
* pyevtk not found, VTK export not available. 
2024-06-19 13:40:57.860 | INFO     | qbanalysis.hyperfine:<module>:28 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-19 13:40:57.928 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_cycleSummed_rot90_quad1_ROI_results_with_FT_NFFT1024_hanningWindow_270717.mat.
2024-06-19 13:40:57.976 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_allCycles_ROIs_with_FTs_NFFT1024_hanningWindow_270717.mat.
2024-06-19 13:40:58.286 | INFO     | qbanalysis.dataset:loadFinalDataset:220 - Processed data to Xarray OK.
2024-06-19 13:40:58.326 | INFO     | qbanalysis.dataset:loadXeProps:71 - Loaded Xe data from /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/dataLocal/Xe_data_table_fixedFractions.ods.

**Xe measured level splittings and the hyperfine constants.**
Statistical uncer

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

## Improved model

- Add exponential state decays + error function (or masking) at t0.
- Add photoionization phenomenologically.

To test:

- trange.
- type of fit.
- adding noise.

In [2]:
# v2 pkg
from qbanalysis.adv_fitting import * 

2024-06-19 13:40:58.378 | INFO     | qbanalysis.basic_fitting:<module>:21 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-19 13:40:58.378 | INFO     | qbanalysis.adv_fitting:<module>:26 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.


In [3]:
# Create model parameters
params = initParams(xeProps)
params

name,value,initial value,min,max,vary
s0,0.28630000,0.2863,0.00000000,1.00000000,True
s1,0.08550000,0.0855,0.00000000,1.00000000,True
s2,0.14110000,0.1411,0.00000000,1.00000000,True
s3,0.22760000,0.2276,0.00000000,1.00000000,True
tau129,5000.00000,5000,0.00000000,inf,True
tau131,5000.00000,5000,0.00000000,inf,True
l2_amp_0,6.24863687,6.248636873496942,-inf,inf,True
l4_amp_0,-5.99561677,-5.9956167686434725,-inf,inf,True
l2_offset_0,-2.70945464,-2.7094546414634513,-inf,inf,True
l4_offset_0,-9.80291079,-9.80291079496373,-inf,inf,True


In [4]:
# Compute advanced model...
calcDict = calcAdvFitModel(params, xePropsFit=xeProps, dataDict=dataDict)

calcDict.keys()

dict_keys(['xData', 'xePropsFit', 'dataDict', 'trange', 'fitFlag', 'returnType', 'modelDict', 'modelDictSum', 'modelDA', 'modelSum', 'dataIn', 'modelIn', 'res', 'iso', 'dataCol', 'decay', 'ionization'])

In [5]:
# xr.concat([decay, decay.sum("Isotope").expand_dims({"Isotope":['sum']})], dim="Isotope")

In [6]:
# Plot model + decay
plotHyperfineModel(calcDict['modelDA']) * plotHyperfineModel(calcDict['decay'])

:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (sum,sum_std)
   .NdOverlay.III :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (decay,decay_std)
   .NdOverlay.IV  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (decay,decay_std)

In [7]:
plotHyperfineModel(calcDict['ionization'],overlay=['l'])

:HoloMap   [ROI,Q]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (Ionization test,Ionization test_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (Ionization test,Ionization test_std)

In [8]:
modelDS = splitUncertaintiesToDataset(calcDict['ionization'].squeeze())
modelDS

dataDS = splitUncertaintiesToDataset(dataIn.squeeze())
dataDS

NameError: name 'dataIn' is not defined

In [9]:
# Calc residuals...

# Basic function
def residual(model,dataIn):
    """
    Calc least squares residual
    """
    res = (model - dataIn)**2  # Returning single value XR only in testing? Issue with dims?
                            # Ah, OK after fixing t-units
    # res = model.values - dataIn.values  # Force to NP, assumes matching size.

    return res


def residualAdv(model,dataIn, dataUn = None):
    
    model.name = 'model'
    modelDS = splitUncertaintiesToDataset(model)
    modelDS = modelDS.assign_coords({'t':modelDS['t'].values.astype(int)})  # Force coords to avoid float comparison issues.
    
    dataIn.name = 'data'
    dataDS = splitUncertaintiesToDataset(dataIn)
    dataDS = dataDS.assign_coords({'t':dataDS['t'].values.astype(int)})
    
    # Uncertainties - use data_std if present, skip if zero.
    # Use passed data if provided
    res = (dataDS['data'] - modelDS['model'])
    thres = 1e-10
    if (dataUn is None) and (dataDS['data_std'].max() < thres):
        pass
    elif dataDS['data_std'].max() > thres:
        res = res/dataDS['data_std']
    else:
        res = res/dataUn
        
    res.name = 'res'
    
    return res, dataDS, modelDS
    # return (dataDS-model) / uncertainty

# dataIn = dataDict['BLMall'].sel({'ROI':0,'l':4}).copy()
dataIn = dataDict['BLMall'].unstack().sel({'l':[2,4]}).copy()
dataUn = dataDict['BLMerr'].unstack().sel({'l':[2,4]}).copy()  # Main data has uncertainties separately currently

# if trange is not None:
#     modelIn = modelIn.sel(t=slice(trange[0],trange[1]))
#     dataIn = dataIn.sel(t=slice(trange[0],trange[1]))

# res = residual(calcDict['ionization'].squeeze(), dataIn.squeeze())
# res.name = 'residual'

# Adv version....
res, dataDS, modelDS = residualAdv(calcDict['ionization'].squeeze(), dataIn.squeeze(), dataUn = dataUn)

In [10]:
dataDS

<xarray.Dataset>
Dimensions:   (ROI: 2, t: 97, l: 2)
Coordinates:
  * ROI       (ROI) int64 0 1
  * t         (t) int64 -70 -60 -50 -40 -30 -20 -10 ... 840 850 860 870 880 890
  * l         (l) int64 2 4
    m         int64 0
Data variables:
    data      (ROI, t, l) float64 0.09006 -0.05167 0.0858 ... 0.6051 0.2423
    data_std  (ROI, t, l) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [11]:
# dataDS.assign_coords({'t':dataDS['t'].values.astype(int)})

In [12]:
# dataDS['data'] - modelDS['model']

In [13]:
res

<xarray.DataArray 'res' (ROI: 2, t: 97, l: 2, m: 1)>
array([[[[ 46.34759345],
         [256.49607612]],

        [[ 51.29385022],
         [154.42366017]],

        [[ 56.71882279],
         [216.7827132 ]],

        [[ 59.03711235],
         [196.47389747]],

        [[ 59.35891554],
         [181.86063094]],

        [[ 83.63528295],
         [226.60612357]],

        [[ 77.18276461],
         [216.4863098 ]],
...
        [[-14.65921378],
         [ 53.71257511]],

        [[-14.44850454],
         [ 71.39241703]],

        [[-22.86218367],
         [ 59.79533869]],

        [[-19.06878362],
         [ 57.23225756]],

        [[-16.8443495 ],
         [ 84.95978354]],

        [[-17.63116983],
         [ 57.40009872]],

        [[-20.57926181],
         [ 50.51155174]]]])
Coordinates:
  * ROI      (ROI) int64 0 1
  * t        (t) int64 -70 -60 -50 -40 -30 -20 -10 ... 840 850 860 870 880 890
  * l        (l) int64 2 4
  * m        (m) int64 0
    Isotope  <U5 'sum'
    Q        int64 0

In [14]:
plotHyperfineModel(res,overlay=['l'])

:HoloMap   [ROI,m]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (res,res_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (res,res_std)

In [15]:
ionTest = ionizationPhenom(params.valuesdict(),decay)
plotHyperfineModel(ionTest, overlay=['l'])

NameError: name 'decay' is not defined

In [20]:
# Test lmfit with new functions...

def calcAdvlmfit(params, trange=[0,1000], **kwargs):
    
    calcDict = calcAdvFitModel(params, xePropsFit=xeProps, dataDict=dataDict)
    
    # dataIn = dataDict['BLMall'].sel({'ROI':0,'l':4}).copy()
    # dataIn = dataDict['BLMall'].unstack().sel({'l':[2,4]}).copy()
    # dataUn = dataDict['BLMerr'].unstack().sel({'l':[2,4]}).copy()  # Main data has uncertainties separately currently

    # if trange is not None:
    #     modelIn = modelIn.sel(t=slice(trange[0],trange[1]))
    #     dataIn = dataIn.sel(t=slice(trange[0],trange[1]))

    # res = residual(calcDict['ionization'].squeeze(), dataIn.squeeze())
    # res.name = 'residual'

    res, dataDS, modelDS = residualAdv(calcDict['ionization'].squeeze(), dataIn.squeeze(), dataUn = dataUn)

    # Optionally set trange
    # NOTE: may also be set in calcBasicFitModel, so should be more careful here!
    if trange is not None:
        res = res.sel(t=slice(trange[0],trange[1]))
        # dataIn = dataIn.sel(t=slice(trange[0],trange[1]))
    
    return res.values

# Set dataIn...
dataIn = dataDict['BLMall'].unstack().sel({'l':[2,4]}).copy()
dataUn = dataDict['BLMerr'].unstack().sel({'l':[2,4]}).copy()  # Main data has uncertainties separately currently

# out = minimize(calcAdvlmfit, params, kws={'xePropsFit':xeProps, 'dataDict':dataDict})  # Working, but not passing data!
out = minimize(calcAdvlmfit, params, kws={'xePropsFit':xeProps, 'dataDict':dataDict, 'dataIn':dataIn, 'dataUn':dataUn})  # Working, but not passing data!

In [21]:
out

In [22]:
from qbanalysis.plots import plotFinalDatasetBLMt
plotOpts = {'width':800}
calcDict = calcAdvFitModel(out.params, xePropsFit=xeProps, dataDict=dataDict)
plotHyperfineModel(calcDict['ionization'],overlay=['ROI']) * plotFinalDatasetBLMt(**dataDict, **plotOpts)

:HoloMap   [Q,l]
   :NdLayout   [l]
      :Overlay
         .NdOverlay.I   :NdOverlay   [ROI]
            :Spread   [t]   (Ionization test,Ionization test_std)
         .NdOverlay.II  :NdOverlay   [ROI]
            :Curve   [t]   (Ionization test,Ionization test_std)
         .NdOverlay.III :NdOverlay   [ROI]
            :Spread   [t]   (BLM per cycle,BLM per cycle_std)
         .NdOverlay.IV  :NdOverlay   [ROI]
            :Curve   [t]   (BLM input 0)

In [23]:
# Check final residuals
res, dataDS, modelDS = residualAdv(calcDict['ionization'].squeeze(), dataIn.squeeze(), dataUn = dataUn)
plotHyperfineModel(res,overlay=['l'])

:HoloMap   [ROI,m]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (res,res_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (res,res_std)

In [35]:
# PLOT TESTING...
from qbanalysis.plots import plotFinalDatasetBLMt
plotOpts = {'width':800}
calcDict = calcAdvFitModel(out.params, xePropsFit=xeProps, dataDict=dataDict)
# plotHyperfineModel(calcDict['ionization'],overlay=['ROI']).layout('l')

# To fix layout issues, treat l separately...
l2 = (plotFinalDatasetBLMt(**dataDict, **plotOpts) * plotHyperfineModel(calcDict['ionization'],overlay=['ROI'])).select(l=2)
l4 = (plotFinalDatasetBLMt(**dataDict, **plotOpts) * plotHyperfineModel(calcDict['ionization'],overlay=['ROI'])).select(l=4)

(l2.overlay('l').opts(title="l2") + l4.overlay('l').opts(title="l4")).cols(1)


:Layout
   .HoloMap.I  :HoloMap   [Q]
      :NdOverlay   [l]
         :Overlay
            .NdOverlay.I   :NdOverlay   [ROI]
               :Spread   [t]   (BLM per cycle,BLM per cycle_std)
            .NdOverlay.II  :NdOverlay   [ROI]
               :Curve   [t]   (BLM input 0)
            .NdOverlay.III :NdOverlay   [ROI]
               :Spread   [t]   (Ionization test,Ionization test_std)
            .NdOverlay.IV  :NdOverlay   [ROI]
               :Curve   [t]   (Ionization test,Ionization test_std)
   .HoloMap.II :HoloMap   [Q]
      :NdOverlay   [l]
         :Overlay
            .NdOverlay.I   :NdOverlay   [ROI]
               :Spread   [t]   (BLM per cycle,BLM per cycle_std)
            .NdOverlay.II  :NdOverlay   [ROI]
               :Curve   [t]   (BLM input 0)
            .NdOverlay.III :NdOverlay   [ROI]
               :Spread   [t]   (Ionization test,Ionization test_std)
            .NdOverlay.IV  :NdOverlay   [ROI]
               :Curve   [t]   (Ionization test,Ionization test_std)

In [36]:
calcDict['xePropsFit']

A/MHz      B/MHz  Splitting/cm−1 label
Isotope I   F   F′                                             
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan        0.291135    s0
131     1.5 1.5 0.5  1697+/-30     -8+/-7        0.085093    s1
            2.5 1.5  1697+/-30     -8+/-7        0.145841    s2
                0.5  1697+/-30     -8+/-7        0.230934    s3

In [38]:
from qbanalysis.basic_fitting import compareResults, extractABParams
compareResults(xeProps,calcDict['xePropsFit'])

original       fit      diff
Isotope I   F   F′                                      
129     0.5 0.5 1.5  0.2863+/-0.0005  0.291135 -0.004835
131     1.5 1.5 0.5  0.0855+/-0.0010  0.085093  0.000407
            2.5 1.5  0.1411+/-0.0029  0.145841 -0.004741
                0.5  0.2276+/-0.0029  0.230934 -0.003334

In [41]:
xePropsFit  =extractABParams(calcDict['xePropsFit'])
xePropsFit.style.set_caption("Updated results")

/opt/conda/lib/python3.10/site-packages/scipy/optimize/_lsq/least_squares.py:111: UserWarning: Setting `gtol` below the machine epsilon (2.22e-16) effectively disables the corresponding termination condition.
  warn("Setting `{}` below the machine epsilon ({:.2e}) effectively "


`gtol` termination condition is satisfied.
Function evaluations 16, initial cost 8.6538e-07, final cost 1.2884e-24, first-order optimality 9.83e-19.


In [42]:
xePropsFit.droplevel(['I','F′','F'])[0:2][['A/MHz','B/MHz']]

,A/MHz,B/MHz
Isotope,,
129,-5818.680912,nan+/-nan
131,1740.852966,40.155254


![Xe table](xe_table.png)

In [ ]:
break

### Testing CI

See https://lmfit.github.io/lmfit-py/confidence.html

Currently lmfit not returning std errors...?

In [48]:
# mini = lmfit.Minimizer(calcAdvlmfit, params, fcn_kws={'xePropsFit':xeProps, 'dataDict':dataDict})  # Working, but not passing data!
# result = minimize()

mini = lmfit.Minimizer(calcAdvlmfit, params, fcn_kws={'xePropsFit':xeProps, 'dataDict':dataDict, 'dataIn':dataIn, 'dataUn':dataUn})  
result = mini.minimize()

In [49]:
print(lmfit.fit_report(result.params))

[[Variables]]
    s0:           0.29113546 (init = 0.2863)
    s1:           0.08509324 (init = 0.0855)
    s2:           0.14110000 (init = 0.1411)
    s3:           0.23093376 (init = 0.2276)
    tau129:       3389.23696 (init = 5000)
    tau131:       3765.50030 (init = 5000)
    l2_amp_0:     1.94105646 (init = 6.248637)
    l4_amp_0:     1.87625695 (init = -5.995617)
    l2_offset_0:  0.47571709 (init = -2.709455)
    l4_offset_0: -0.01934226 (init = -9.802911)
    l2_amp_1:    -1.80547603 (init = 6.750814)
    l4_amp_1:    -1.91628564 (init = 4.770826)
    l2_offset_1:  0.57907158 (init = 3.059341)
    l4_offset_1:  0.15870698 (init = -3.912131)


In [47]:
ci = lmfit.conf_interval(mini, result)
lmfit.printfuncs.report_ci(ci)

MinimizerException: Cannot determine Confidence Intervals without sensible uncertainty estimates

In [50]:
result.params

name,value,initial value,min,max,vary
s0,0.29113546,0.2863,0.00000000,1.00000000,True
s1,0.08509324,0.0855,0.00000000,1.00000000,True
s2,0.14110000,0.1411,0.00000000,1.00000000,True
s3,0.23093376,0.2276,0.00000000,1.00000000,True
tau129,3389.23696,5000,0.00000000,inf,True
tau131,3765.50030,5000,0.00000000,inf,True
l2_amp_0,1.94105646,6.248636873496942,-inf,inf,True
l4_amp_0,1.87625695,-5.9956167686434725,-inf,inf,True
l2_offset_0,0.47571709,-2.7094546414634513,-inf,inf,True
l4_offset_0,-0.01934226,-9.80291079496373,-inf,inf,True


In [51]:
# Try forcing std errors, per https://lmfit.github.io/lmfit-py/confidence.html#working-without-standard-error-estimates
for p in result.params:
    result.params[p].stderr = abs(result.params[p].value * 0.1)

In [52]:
ci = lmfit.conf_interval(mini, result)
lmfit.printfuncs.report_ci(ci)

/opt/conda/lib/python3.10/site-packages/lmfit/confidence.py:330: UserWarning: rel_change=0.0 < 1e-05 at iteration 2 and prob(s2=0.11288) = 2.209960073191247e-07 < max(sigmas)
  warn(errmsg)
/opt/conda/lib/python3.10/site-packages/lmfit/confidence.py:330: UserWarning: rel_change=0.0 < 1e-05 at iteration 2 and prob(s2=0.16932000000000003) = 2.209960073191247e-07 < max(sigmas)
  warn(errmsg)


                99.73%    95.45%    68.27%    _BEST_    68.27%    95.45%    99.73%
 s0         :  -0.00075  -0.00050  -0.00025   0.29114  +0.00025  +0.00050  +0.00075
 s1         :  -0.00257  -0.00170  -0.00084   0.08509  +0.00084  +0.00167  +0.00251
 s2         :      -inf      -inf      -inf   0.14110      +inf      +inf      +inf
 s3         :  -0.00203  -0.00135  -0.00067   0.23093  +0.00067  +0.00134  +0.00201
 tau129     :-1547.59078-1209.38714-731.657073389.23696+1271.60146+4057.65132+15212.35683
 tau131     :-1990.40889-1590.38286-994.120763765.50030+2013.85978+8365.30324      +inf
 l2_amp_0   :  -0.43387  -0.28966  -0.14528   1.94106  +0.14688  +0.29608  +0.44834
 l4_amp_0   :  -0.20877  -0.13972  -0.07023   1.87626  +0.07126  +0.14386  +0.21814
 l2_offset_0:  -0.02302  -0.01530  -0.00763   0.47572  +0.00764  +0.01531  +0.02306
 l4_offset_0:  -0.00507  -0.00337  -0.00168  -0.01934  +0.00168  +0.00337  +0.00507
 l2_amp_1   :  -0.80288  -0.52744  -0.26039  -1.80548  +0.25538  +0

### TEST FIT 1

In [17]:
out

In [18]:
from qbanalysis.plots import plotFinalDatasetBLMt
plotOpts = {'width':800}
calcDict = calcAdvFitModel(out.params, xePropsFit=xeProps, dataDict=dataDict)
plotHyperfineModel(calcDict['ionization'],overlay=['ROI']) * plotFinalDatasetBLMt(**dataDict, **plotOpts)

:HoloMap   [Q,l]
   :NdLayout   [l]
      :Overlay
         .NdOverlay.I   :NdOverlay   [ROI]
            :Spread   [t]   (Ionization test,Ionization test_std)
         .NdOverlay.II  :NdOverlay   [ROI]
            :Curve   [t]   (Ionization test,Ionization test_std)
         .NdOverlay.III :NdOverlay   [ROI]
            :Spread   [t]   (BLM per cycle,BLM per cycle_std)
         .NdOverlay.IV  :NdOverlay   [ROI]
            :Curve   [t]   (BLM input 0)

In [ ]:
break

In [19]:
# Check final residuals
res, dataDS, modelDS = residualAdv(calcDict['ionization'].squeeze(), dataIn.squeeze(), dataUn = dataUn)
plotHyperfineModel(res,overlay=['l'])

:HoloMap   [ROI,m]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (res,res_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (res,res_std)

In [ ]:
plotFinalDatasetBLMt(**dataDict, **plotOpts)

In [ ]:
# # Functions for calculation.
# # NOTE: designed for use with lmfit, expect lmfit Parameters() or params dict.

# import lmfit
# from qbanalysis.basic_fitting import calcBasicFitModel


# def calcDecays(paramDict, isoDA):
#     """
#     Apply exponential decays, exp(-t/tau), per isotope.
    
#     paramDict : dict
#         Parameters dictionary with items 'tauZ' per isotope Z.
        
#     isoDA : xr.dataarray
#         Main data structure, with dims including "Isotope" and "t".
#         Groupby & apply exponential decay per Isotope.
        
#     """
    
#     #*** Apply exponential decay (per isotope)
#     # Easier way to do this...? Here set DA per isotope, then combine and multiply.
#     # Quick test of multiply without all isotope dims also awkward, although maybe groupby would work...
    
#     # Dict + loop version
#     # decay = xr.zeros_like(isoDA)
# #     decayDict = {}
# #     for iso in [129,131]:
# #         decayDict[iso] = np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
# #         # decay = decay + np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
    
# #     decayDA = isoDA*stackModelToDA(decayDict)
    
#     # Groupby version - better...?
#     # Test groupby...
#     decay = isoDA.groupby("Isotope").map(lambda x: x*np.exp(-x.t/paramDict[f"tau{x.Isotope.values.item().rstrip('Xe')}"]))
#     decay.name = 'decay'
    
#     return decay

# def ionizationPhenom(paramDict,modelDA):
#     """
#     Basic amplitude/phase + offset ionization channel model.

#     """
    
#     # Assume isotope independent...?
#     # Also ignore total XS here
#     modelIn = modelDA.sum("Isotope").sel({'K':2})
    
#     # ROI only version - assume isotope independent params
#     modelOutComponents = []
#     for ROI in [0,1]:
#         lparams = [2,4]
#         paramsAmp = [paramDict[f"l{l}_amp_{ROI}"] for l in lparams]
#         paramsOffset = [paramDict[f"l{l}_offset_{ROI}"] for l in lparams]
        
#         modelOutComponents.append((modelIn * xr.DataArray(paramsAmp,coords=[("l",lparams)]) + xr.DataArray(paramsOffset,coords=[("l",lparams)])).expand_dims({"ROI":[ROI]}))
        
#         # for item in ['amp','offset']:
#         #     [params.add(f"l{l}_{item}_{ROI}", value=1.0) for l in [2,4]]
    
#     modelOut = xr.concat(modelOutComponents,dim="ROI")
#     modelOut.name = "Ionization test"
    
#     # K2t = decay.sel({'K':2,'Isotope':'129Xe'}).squeeze()
#     # K2t = K2t * xr.DataArray([20,-10],coords=[("l",[0,2])]) + xr.DataArray([1,-1],coords=[("l",[0,2])])
#     # K2t.name = "Ionization test"

#     return modelOut
                                           

# def calcAdvFitModel(params, **kwargs):
#     """
#     Wrap basic fit model, and add some features.
    
#     Note passed params expect lmfit Parameters() object, or dictionary.
    
#     """
    
#     if isinstance(params, lmfit.parameter.Parameters):
#         paramDict = params.valuesdict()
#     else:
#         paramDict = params
    
#     #*** Run basic case as base
#     # TODO: return modelDA here too
#     # TODO: arg passing
#     # xDataBasic = xData[0:4]  # For basic case only use first 4 (splitting) params
#     # xDataBasic = [paramDict[k] for k,v in paramDict.items() if k.startswith('s')]  # For dict case, use s0...3
#     xDataBasic = [paramDict[f"s{n}"] for n in range(0,4)]  # Assume number of params and enforce ordering. 
#     calcBasicDict = calcBasicFitModel(xDataBasic, fitFlag=False, returnType='full', **kwargs)
    
#     # Use original model results and apply additional params
#     # modelDA = stackModelToDA(calcDict['modelDict'])  # Use original model?
#     isoDA = calcBasicDict['modelDA'].sel({'Isotope':['129Xe','131Xe']})  # Use isotope-weighted results
    
    
#     #*** Apply exponential decay (per isotope)
#     # Easier way to do this...? Here set DA per isotope, then combine and multiply.
#     # Quick test of multiply without all isotope dims also awkward, although maybe groupby would work...
    
#     # Dict + loop version
#     # decay = xr.zeros_like(isoDA)
# #     decayDict = {}
# #     for iso in [129,131]:
# #         decayDict[iso] = np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
# #         # decay = decay + np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
    
# #     decayDA = isoDA*stackModelToDA(decayDict)
    
#     # Groupby version - better...?
#     # Test groupby...
#     # decay = isoDA.groupby("Isotope").map(lambda x: x*np.exp(-x.t/paramDict[f"tau{x.Isotope.values.item().rstrip('Xe')}"]))
#     # decay.name = 'decay'
    
#     decay = calcDecays(paramDict, isoDA)

#     #*** Apply ionization model
#     # 
    
#     return calcBasicDict, decay
    

In [ ]:
plotOpts = {'width':800}
fitParamsCol = 'Splitting/cm−1'
xePropsFit = xeProps.copy()
x0 = unumpy.nominal_values(xePropsFit[fitParamsCol].to_numpy())

# calcDict = calcAdvFitModel(x0, xePropsFit=xePropsFit, dataDict=dataDict)
calcDict, decay = calcAdvFitModel(params, xePropsFit=xePropsFit, dataDict=dataDict)


In [ ]:
calcDict.keys()

In [ ]:
decay

In [ ]:
# Ionization... just want amplitude + offset per channel...?

# decay.sel({'K':2}).groupby('Isotope').apply(

# Test single channel... OK
K2t = decay.sel({'K':2,'Isotope':'129Xe'}).squeeze()
K2t = K2t * xr.DataArray([20,-10],coords=[("l",[0,2])]) + xr.DataArray([1,-1],coords=[("l",[0,2])])
K2t.name = "Ionization test"

dataPlot = K2t
overlay = 'l'

DS = splitUncertaintiesToDataset(dataPlot)
hvDS = hvPlotters.hv.Dataset(DS.unstack())
hvDS.to(hvPlotters.hv.Curve, kdims = ['t']).overlay(overlay)
# plotHyperfineModel(K2t, plotSpread=False, overlay=['l'])  # NOTE - now debugged!

In [ ]:
plotHyperfineModel(isoDA) * plotHyperfineModel(decay.sum('Isotope'))

In [ ]:
ionTest = ionizationPhenom(params.valuesdict(),decay)
plotHyperfineModel(ionTest, overlay=['l'])

In [ ]:
ionTest

In [ ]:
decay

In [ ]:
ionizationPhenom(params.valuesdict(),decay)

In [ ]:
isoDA = calcDict['modelDA'].sel({'Isotope':['129Xe','131Xe']})

In [ ]:
isoDA

In [ ]:
decay = xr.ones_like(isoDA)

dTemp = {}

for iso in [129,131]:
    # # decay = decay.sel({"Isotope":f"{iso}Xe"}, drop=False) * np.exp(-isoDA.t/paramDict[f"tau{iso}"])

    dTemp[iso] = np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
    
    

# decay.where(
# decay = decay*dTemp

# decay.sel({"Isotope":"129Xe"}) * dTemp + decay.sel({"Isotope":"131Xe"})
# decay


decayStack = stackModelToDA(dTemp)

isoDA*decayStack

In [ ]:
isoDA.Isotope.values.item()  #.rstrip('Xe')

In [ ]:
# Test groupby...
isoDA.groupby("Isotope").map(lambda x: np.exp(-x.t/paramDict[f"tau{x.Isotope.values.item().rstrip('Xe')}"]))
# isoDA.groupby("Isotope").map(lambda x: print(x.Isotope.values.item().rstrip('Xe')))

In [ ]:
from scipy.special import erf  # Error function if required, or use as mask?

tau=500
# decay = erf(isoDA.t)*np.exp(-isoDA.t/tau).expand_dims({'Isotope':['129Xe']})  # Just add isotope dim to allow * by main data?
decay = np.exp(-isoDA.t/tau).expand_dims({'Isotope':['129Xe']})  # Just add isotope dim to allow * by main data?


In [ ]:
# Basics OK.
# TODO: should multiply only K=0 and renorm?
# TODO: remove t<=0 terms.
testMult = isoDA * decay

testMult.values = unumpy.nominal_values(testMult)

# xr.where(testMult.t>0, unumpy.nominal_values(testMult),0)

testMult.squeeze().unstack().hvplot.line(x='t')

In [ ]:
erf(isoDA.t)

## Fitting with lmfit

- Easy parameter control.
- Propagate uncertainties.

See https://lmfit.github.io/lmfit-py/intro.html

In [ ]:
xeProps

In [ ]:
# def pdIndexMap(df):
#     """
#     Convert list of tuple labels to short str format from PD dataframe.
    
#     Also append short names as column in df.
    
#     Useful for setting up mappings to fitting params for lmfit.
#     """
    
#     indList = df.index
    
#     # Parameter names for lmfit [a-z_][a-z0-9_]*, so replace '.' and '-' signs.
#     nameList = ['_'.join(str(ele).replace('-','n').replace('.','') for ele in sub) for sub in indList]
    
#     # shortNameList = [list(f"{str(ele)}_s{n}" for ele in sub) for n,sub in enumerate(indList)]
#     # shortNameList = [f"I{str(sub[0])}_s{n}" for n,sub in enumerate(indList)]
#     shortNameList = [f"s{n}" for n,sub in enumerate(indList)]
    
#     # Generate map {full names : lables}
#     indMapLong = dict(zip(indList, nameList))
#     indMapShort = dict(zip(indList, shortNameList))
    
#     # Append to original table
#     df['label']=shortNameList
    
#     return locals()


# def pdParamsReplaceFromMap(df, pdMap, params, dataCol = 'Splitting/cm−1'):
#     """
#     Convert lmfit.params items back to original PD dataframe via labels.
#     """

#     dfOut = df.copy()

#     for k,v in pdMap['indMapShort'].items():
#         # Replace by keys
#         # xeTest.loc[k][dataCol] = xeTest.loc[k][dataCol]*n

#         # Replace by value (short name) lookup
#         # This should ensure consistency in replacement ordering etc.
#         dfOut.loc[dfOut['label']==v,dataCol] = params.valuesdict()[v]
        
#         # print(params.valuesdict()[v])


#     return dfOut

In [ ]:
pdMap = pdIndexMap(xeProps)
# xeProps['lmfit']=pdMap['shortNameList']
xeProps
# pdMap

In [ ]:
# Create parameters for the fit
# v2 better
# from lmfit import minimize, Parameters

# def initParams(xeProps):
#     """
#     Init lmfit Parameters() for Xe advanced hyperfine model
    
#     """
    
#     # Set labels for params
#     pdMap = pdIndexMap(xeProps)

#     # Setup parameters from df
#     params = Parameters()

#     # Iterate over PD rows and assign to params.
#     # May be neater way to do this...?
#     for item in xeProps.iterrows():
#         itemVal = unumpy.nominal_values(item[1]['Splitting/cm−1'])
#         params.add(item[1]['label'], value = itemVal, min = 0, max = 1)

#     #*** Add additional params as required for the advanced model...
#     # Lifetimes
#     params.add("tau129", value = 500)
#     params.add("tau131", value = 500)

#     # Ionization model params - just set amplitude + offset for l=2,4 modelling
#     # Will also need ROI (channel) here too...? Or just apply these to sum, if assumed iso independent
#     # for iso in [129,131]:
#     #     for item in ['amp','offset']:
#     #         [params.add(f"l{l}_{item}_{iso}", value=1.0) for l in [2,4]]

#     # ROI only version - assume isotope independent params
#     for ROI in [0,1]:
#         for item in ['amp','offset']:
#             [params.add(f"l{l}_{item}_{ROI}", value=np.random.uniform(-1,1)) for l in [2,4]]

#     return params

params = initParams(xeProps)
params

In [ ]:
dataDict

In [ ]:
type(params)  #['s0']   #.valuesdict()['s0']

In [ ]:
# Test param remapper
pdParamsReplaceFromMap(xeProps,pdMap, params)

In [ ]:
params.valuesdict()

In [ ]:
# Check unpacking - ordering OK...?
paramDict = params.valuesdict()
[paramDict[k] for k,v in paramDict.items() if k.startswith('s')] 

In [ ]:
# Enforce...
[paramDict[f"s{n}"] for n in range(0,4)] 

In [ ]:
dataCol = 'Splitting/cm−1'
# [xeProps.loc[k][dataCol]*2 for k,v in pdMap['indMapShort'].items()]

xeTest = xeProps.copy()
n=1
for k,v in pdMap['indMapShort'].items():
    # Replace by keys
    # xeTest.loc[k][dataCol] = xeTest.loc[k][dataCol]*n
    
    # Replace by value (short name) lookup
    # This should ensure consistency in replacement
    xeTest.loc[xeTest['label']==v,dataCol] = params.valuesdict()[v]
    
    n = n+5
    
xeTest
    



In [ ]:
xeProps

In [ ]:
# Create parameters for the fit
# v1 testing
from lmfit import minimize, Parameters

params = Parameters()

# Iterate over PD rows and assign to params.
# May be neater way to do this...?
for item in xeProps.iterrows():
    # print(item)
    
    # Name from index - this 
    # itemInds = ''.join((str(value) for value in item[0]))
    itemInds = [''.join(str(value)) for value in item[0]]
    itemName = '_'.join(itemInds).replace('.','')  # Set param name from indexes
                                                   # Note "[a-z_][a-z0-9_]*" and valid python var name.
    
    # print(itemName)
    # print(type(itemName))
    # print((''.join(str(value)) for value in item[0]))
    # list(item[0]).join('')
    # params.add(f"Splitting_{itemName}", value = item[-1][0], min = 0, max = 1)
    
    # OK up to itemName[0:5]
    # params.add(f"s_{itemName[0:5]}", value = item[-1][0], min = 0, max = 1)
    # params.add(f"s{itemName[0:7]}", value = item[-1][0], min = 0, max = 1)
    
    # print(item[-1])  #['Splitting/cm−1'])
    itemVal = unumpy.nominal_values(item[-1]['Splitting/cm−1'])
    params.add(f"s_{itemName}", value = itemVal, min = 0, max = 1)

# params.add('amp', value=10, vary=False)
# params.add('decay', value=0.007, min=0.0)
# params.add('phase', value=0.2)
# params.add('frequency', value=3.0, max=10)
                            
params

# TODO: easy/neat way to index these back to original case... see PEMtk....
# UPDATE: main difference there is piece-wise list/dict creation prior to setting params, but quite similar.

In [ ]:
# Map items to names
xeProps.index.to_list()

In [ ]:
params.keys()

In [ ]:
params.valuesdict()

In [ ]:
def testLocals(**kwargs):
    
    z=2
    
    return locals()

testLocals(x=5)

## lmfit reboot/test

From https://lmfit.github.io/lmfit-py/intro.html

In [ ]:
from numpy import linspace, random
from scipy.optimize import leastsq

# generate synthetic data with noise
x = linspace(0, 100)
noise = random.normal(size=x.size, scale=0.2)
data = 7.5 * sin(x*0.22 + 2.5) * exp(-x*x*0.01) + noise

# generate experimental uncertainties
uncertainty = abs(0.16 + random.normal(size=x.size, scale=0.05))

In [ ]:
from numpy import exp, sin

from lmfit import minimize, Parameters


def residual(params, x, data, uncertainty):
    amp = params['amp']
    phaseshift = params['phase']
    freq = params['frequency']
    decay = params['decay']

    model = amp * sin(x*freq + phaseshift) * exp(-x*x*decay)

    return (data-model) / uncertainty


params = Parameters()
params.add('amp', value=10)
params.add('decay', value=0.007)
params.add('phase', value=0.2)
params.add('frequency', value=3.0)

out = minimize(residual, params, args=(x, data, uncertainty))

In [ ]:
out

## Test some PEMtk methods...

In [ ]:
from pemtk.fit._util import lmmuListStrReformat

lmmuListStrReformat(xeProps.index)